In [62]:
# Multilayer perceptron (Keras)

In [1]:
import pandas as pd
import numpy as np
from statsmodels.robust.scale import mad
import os

In [2]:
def mergeData():
    SaveFile_Name = 'data.csv'
    file_list = os.listdir('data')
    df = pd.read_csv('data'+'\\'+file_list[0])
    df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv('data'+'\\'+file_list[i])
        df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

In [3]:
def readData():
  train = pd.read_csv("data.csv")
  return train

In [4]:
def changeYear(data):
    for i in range(0,data.shape[0]):
        Date=data["日期"][i].split('/')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        data.loc[i,"日期"]=year+'/'+month+'/'+date
    return data

In [5]:
def augFeatures(data):
  data["日期"] = pd.to_datetime(data["日期"])
  data["年"] = data["日期"].dt.year
  data["月"] = data["日期"].dt.month
  data["日"] = data["日期"].dt.day
  data["第幾日"] = data["日期"].dt.dayofweek
  return data

In [6]:
def normalizeFunction(x):
    return ((x - np.mean(x)) / (np.max(x) - np.min(x)))

In [19]:
def normalize(data):
    for i in range(0,data.shape[0]):
        if data["漲跌價差"][i]=='X0.00':
            data.loc[i,"漲跌價差"]=str(int(data["收盤價"][i])-int(data["收盤價"][i-1]))
    data=data.drop(["日期"], axis=1)
    data=data.drop(["成交股數"], axis=1)
    data=data.drop(["成交金額"], axis=1)
    #data=data.drop(["漲跌價差"], axis=1)
    data=data.drop(["成交筆數"], axis=1)
    datanormalize=data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return datanormalize

In [20]:
mergeData()
getdata=readData()
changedata=changeYear(getdata)
adddata=augFeatures(changedata)
train=normalize(adddata)
print(train)

TypeError: ('Could not convert 6.06.57.07.58.08.59.09.510.011.0-2.00.02.0-4.06.0-4.0-1.01.01.0-1.01.00.02.03.0-10.02.0-11.02.08.00.00.01.00.0-1.0-3.01.0-2.0-2.0-2.0-1.0-3.03.03.0-4.01.0-1.02.0-1.0-1.00.03.01.00.00.00.0-1.0-1.00.02.0-1.0-1.0-1.00.02.0-1.0-1.01.0-1.01.0-1.07.0-2.00.0-2.0-1.0-2.00.0-2.0-1.00.0-1.01.02.0-1.0-1.01.0-2.00.00.01.00.00.01.0-2.0-1.00.0-3.0-1.0-0.5-1.5-2.0-6.0-2.5-0.51.56.5-3.55.0-6.5-1.04.0-2.02.05.0-0.5-1.00.51.51.5-1.0-0.53.54.02.0-2.0-3.03.02.0-1.03.01.0-5.0-1.0-7.03.02.02.01.03.0-2.02.01.00.03.0-2.03.00.02.0-2.02.0-1.00.01.00.04.03.02.010.03.0-2.04.0-4.0-1.01.01.01.0-3.0-5.0-5.01.0-4.01.0-1.0-1.02.03.0-2.0-1.00.01.00.00.0-1.00.0-1.0-8.03.02.05.00.02.00.08.0-1.00.0-1.01.0-1.01.0-1.02.01.02.0-2.0-4.0-4.03.07.01.0-2.03.01.02.02.0-1.01.02.0-6.0-2.54.0-2.5-1.50.50.53.02.03.07.5-2.03.04.03.5-8.02.50.50.51.0-3.0-0.51.5-2.03.0-2.0-1.0-4.50.5-3.5-3.50.51.0-1.5-7.00.57.0-0.5-2.0-2.0-3.0-4.53.51.0-3.51.51.0-2.52.0-1.54.5-2.51.55.51.51.0-1.00.01.5-2.5-3.50.00.51.51.00.

In [34]:
train = pd.read_csv("data.csv")
train=train.drop(["日期"], axis=1)
for i in range(0,train.shape[0]):
    if train["漲跌價差"][i]=='X0.00':
        train.loc[i,"漲跌價差"]='-10.00'
train=train.convert_objects(convert_numeric=True)
print(train.dtypes)

成交股數     object
成交金額     object
開盤價     float64
最高價     float64
最低價     float64
收盤價     float64
漲跌價差    float64
成交筆數    float64
dtype: object


C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
